This notebook is for prototyping subnetwork flow.

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dataclasses

import matplotlib.pyplot as plt
import numpy as np
import torch
import transformer_lens.utils as tl_utils
from tqdm.auto import tqdm
from transformer_lens import ActivationCache, HookedTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

from pii import utils

torch.set_grad_enabled(False)

In [5]:
from pii.subnet_flow.masked_model import MaskedTransformer

### Load models

In [3]:
# You will need to login to huggingface first:
#   huggingface-cli login
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
hf_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    low_cpu_mem_usage=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
tl_model = HookedTransformer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    hf_model=hf_model,
    device="cuda",
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    tokenizer=tokenizer,
)
tl_model.generate("The capital of Germany is", max_new_tokens=20, temperature=0)

Using pad_token, but it is not set yet.


Loaded pretrained model meta-llama/Llama-2-7b-chat-hf into HookedTransformer


  0%|          | 0/20 [00:00<?, ?it/s]

'The capital of Germany is Berlin. Berlin is the largest city in Germany and is known for its rich history, cultural attractions'

In [31]:
mtf = MaskedTransformer(tl_model)
for mask in mtf.masks.values():
    mask *= 0

print(mtf.transformer.generate("The capital of Germany is", max_new_tokens=20, temperature=0))
mtf.masks_active = False
print(mtf.transformer.generate("The capital of Germany is", max_new_tokens=20, temperature=0))
# [n for n, p in mtf.named_parameters() if p.requires_grad]

# The above was just a test, we reconstruct the model from scratch here
mtf = MaskedTransformer(tl_model)

  0%|          | 0/20 [00:00<?, ?it/s]

The capital of Germany isntautoritéiséeonnaexitdia⊙nawanrack sjö &=\ife Према obraidorunasabstract Maze


  0%|          | 0/20 [00:00<?, ?it/s]

The capital of Germany is Berlin. Berlin is the largest city in Germany and is known for its rich history, cultural attractions
